In [1]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

In [2]:
import argparse 

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'val/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

In [17]:
import matplotlib.pyplot as plt

def plot_history(history):
    log.info("History keys: %s", history.history.keys())
    # Accuracy
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(history.history['accuracy'], label='Train')
    ax.plot(history.history['val_accuracy'], label='Test')
    ax.set_title('Model accuracy')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Epoch')
    ax.grid(True)
    ax.legend(['Train', 'Val'], loc='lower right')
    
    # Loss
    plt.figure(figsize=(10,5))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.grid()
    plt.show()
    
def build_dataset(data_folder):
#     p1 = pd.read_csv(args.train_folder + 'chorale_000.csv')
    dataset = tf.data.Dataset.list_files(data_folder + 'chorale_0*.csv', seed=args.seed)
    dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1))
#     dataset.map(preprocess)
    dataset = dataset.shuffle(10000)
    dataset = dataset.batch(args.batch_size).prefetch(1)
    for d in dataset:
        print(d)
        break
    
#     for d in dataset:
#         print(d)
#         break
#     return dataset

def baseline_model():
    model = keras.models.Sequential()
    model.add(keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 4]))
    model.add(keras.layers.SimpleRNN(20)
    model.add(keras.layers.Dense(10))
    return model

build_dataset(data_folder=args.train_folder)

tf.Tensor(
[b'67,64,59,52' b'69,65,57,50' b'74,67,59,55' b'69,64,60,45'
 b'65,60,57,41' b'74,68,65,47' b'70,62,55,43' b'71,67,62,50'
 b'70,62,53,46' b'72,67,60,45' b'72,64,57,55' b'69,62,57,54'
 b'71,55,62,54' b'72,67,60,52' b'72,60,64,57' b'70,67,62,46'
 b'69,63,60,48' b'69,62,59,53' b'65,60,57,57' b'69,65,60,53'
 b'71,62,59,55' b'74,65,57,53' b'69,66,60,50' b'78,69,62,48'
 b'67,62,59,43' b'69,66,60,50' b'67,63,60,48' b'71,66,62,56'
 b'63,63,55,39' b'67,62,59,54' b'74,62,57,54' b'71,62,59,55'], shape=(32,), dtype=string)
